In [1]:
import vtk
import ipywidgets as widgets
from IPython.display import display

# Read data from an XML image file
reader = vtk.vtkXMLImageDataReader()
reader.SetFileName("./mixture.vti")
reader.Update()

# Extract data from the reader
data = reader.GetOutput()

In [2]:
# Initialize lists to store coordinates and scalar values
x=[]
y=[]
z=[]
values=[]

In [3]:
# iterate thruogh the data to extract coordinates and scalar values
for i in range(0,75):
    for j in range(0,75):
        for k in range(0,75):
            x.append(i)
            y.append(j)
            z.append(k)
            values.append(data.GetScalarComponentAsDouble(i,j,k,0))

In [4]:
import plotly.graph_objects as go
initial_isovalue=0
fig= go.FigureWidget(data=go.Isosurface(        # create a Plotly figure widget for the isosurface plot
    x=x,
    y=y,
    z=z,
    value=values,
    colorscale='plasma',
    showscale=False,
    isomin=initial_isovalue,
    isomax=initial_isovalue,
    cmin=min(values),
    cmax=max(values)
))




In [5]:
# Create slider to control the isovalue
slider = widgets.FloatSlider(
    value=initial_isovalue,
    min=min(values),
    max=max(values),
    step=0.01,
    description='Isovalue:',
    continuous_update=False
)


In [6]:
# update layout settings for the 3D scene
fig.update_layout(scene=dict(
        xaxis=dict(title='X', showticklabels=False),
        yaxis=dict(title='Y', showticklabels=False),
        zaxis=dict(title='Z', showticklabels=False),
        aspectmode='cube',  # Ensure isotropic scaling
        # camera=dict(
        #     eye=dict(x=1.5, y=1.5, z=1.5)  # Set initial camera position
        # )
    ))

FigureWidget({
    'data': [{'cmax': 0.43280163407325745,
              'cmin': -0.9935540556907654,
              'colorscale': [[0.0, '#0d0887'], [0.1111111111111111, '#46039f'],
                             [0.2222222222222222, '#7201a8'], [0.3333333333333333,
                             '#9c179e'], [0.4444444444444444, '#bd3786'],
                             [0.5555555555555556, '#d8576b'], [0.6666666666666666,
                             '#ed7953'], [0.7777777777777778, '#fb9f3a'],
                             [0.8888888888888888, '#fdca26'], [1.0, '#f0f921']],
              'isomax': 0,
              'isomin': 0,
              'showscale': False,
              'type': 'isosurface',
              'uid': '0b93b120-4ce6-479d-9488-c994bbb11b0e',
              'value': [-0.04087147116661072, -0.020417775958776474,
                        -0.004733099602162838, ..., -0.9426848292350769,
                        -0.8584827780723572, -0.7285129427909851],
              'x': [0, 0, 0, .

In [7]:
# Create reset button
reset_button = widgets.Button(description='Reset')
# Define function to reset isovalue
def reset_isovalue(button):
    slider.value = 0.0
# Link reset button and reset function
reset_button.on_click(reset_isovalue)

In [8]:
# Define function to update isosurface
def update_isosurface_histogram(change):
    fig.data[0].isomin = change.new
    fig.data[0].isomax = change.new
    isovalue = change.new
    selected_data = initial_hist_data if isovalue == 0 else [x for x in initial_hist_data if (fig.data[0].isomin - 0.25) <= x <= (fig.data[0].isomin + 0.25)]
    hist.data[0].x = selected_data
# Link slider and update function
slider.observe(update_isosurface_histogram, names='value')



In [9]:
from plotly.subplots import make_subplots
import numpy as np
flat_values = np.array(values).flatten()

In [10]:
# Create initial histogram
initial_hist_data = flat_values
hist = go.FigureWidget(data=[go.Histogram(
    x=initial_hist_data,
    nbinsx=30,
    marker=dict(color='blue'),
    name='Value Distribution',
)])
hist.update_layout(yaxis=dict(title='Frequency'))
hist.update_layout(xaxis=dict(title='Vortex scalar values'))

FigureWidget({
    'data': [{'marker': {'color': 'blue'},
              'name': 'Value Distribution',
              'nbinsx': 30,
              'type': 'histogram',
              'uid': '8f9ca5eb-876f-4cd8-8fa1-d9f45f189e90',
              'x': array([-0.04087147, -0.02041778, -0.0047331 , ..., -0.94268483, -0.85848278,
                          -0.72851294])}],
    'layout': {'template': '...',
               'xaxis': {'title': {'text': 'Vortex scalar values'}},
               'yaxis': {'title': {'text': 'Frequency'}}}
})

In [11]:
fig.layout.width = 500  # Set the width to 800 pixels
fig.layout.height = 500  # Set the height to 600 pixels

# Customize the layout of the Histogram (hist)
hist.layout.width = 500  # Set the width to 800 pixels
hist.layout.height = 400  # Set the height to 300 pixels

In [12]:
# Display slider and isosurface
display(widgets.VBox([widgets.HBox([slider, reset_button]), widgets.HBox([fig,hist])]))